In [2]:
from rdkit.Chem import Descriptors
from sklearn.preprocessing import scale
from mendeleev.fetch import fetch_ionization_energies, fetch_table
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
from torch_geometric.data import DataLoader
from rdkit import Chem
from tqdm import tqdm

from src.network import *
from src.train import *
from src.utility import *
from src.evaluate import *


In [4]:
train_csv, test_csv, submission_csv = load_dataset()

In [5]:
train_csv

,uid,SMILES,S1_energy(eV),T1_energy(eV),ST1_GAP(eV)
0,train_0,CCC1CCCCN1C(=O)C(C)OC(=O)c1c(C)oc(-n2cccc2)c1C#N,4.6747,3.3809,1.2938
1,train_1,COc1ccc(Oc2ccc(N3C(=S)NC(c4ccccn4)C3c3cc(C)n(-...,3.6617,3.4585,0.2032
2,train_2,CC(=O)Nc1ccc(C(=O)[C@H](C)Sc2nnc(C3CCCCC3)o2)cc1,3.6420,3.1787,0.4633
3,train_3,OC(CNC1CC1)CN1CCc2sccc2C1,4.8901,3.7847,1.1054
4,train_4,CCNC(CCCC(F)(F)F)C1(OCC)CCOCC1,6.4967,6.2724,0.2243
...,...,...,...,...,...
30340,dev_66,N#Cc1cc(-c2ccc(N3c4ccccc4Oc4ccccc43)cc2)c(-c2c...,2.1939,2.1846,0.0093
30341,dev_67,CC1(C)c2ccccc2N(c2ccc(-c3nc4ccc(N5c6ccccc6C(C)...,2.3537,2.3371,0.0166
30342,dev_68,Cc1cc(-n2c3ccc(C(C)(C)C)cc3c3cc(C(C)(C)C)ccc32...,2.1364,2.1260,0.0104
30343,dev_69,Cc1cc(-n2c3ccccc3c3ccccc32)cc(C)c1B1c2ccccc2B(...,2.2650,2.2511,0.0139


In [22]:
mol = Chem.MolFromSmiles(train_csv.iloc[0]["SMILES"])
fp = Chem.RDKFingerprint(mol)

from rdkit import DataStructs

arr = np.zeros((0,))
DataStructs.ConvertToNumpyArray(fp, arr)
arr[0:100]

array([1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1.,
       1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
       0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1.,
       0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 1.,
       1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0.])

array([1., 1., 1., ..., 1., 1., 1.])